### Origin of the data

The data used for the queries in the following sections come from orders dataset. The data in the dataset follow the schema:

#### 1. Find the number of distinct products

In [ ]:
%%sparksql
SELECT COUNT(*)
FROM (
    SELECT DISTINCT i.product
    FROM orders
    LATERAL VIEW EXPLODE(items) AS i
)

#### 2. Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. 

In [ ]:
%%sparksql
SELECT i.product AS product, AVG(i.quantity) AS avg
FROM orders
LATERAL VIEW EXPLODE(items) AS i
GROUP BY product
ORDER BY avg DESC
LIMIT 10

#### 3. Find the most expensive order

In [ ]:
%%sparksql
SELECT order_id, SUM(i.price * i.quantity) AS p
FROM orders
LATERAL VIEW EXPLODE(items) AS i
GROUP BY order_id
ORDER BY p DESC
LIMIT 1

### Origin of the data

The next queries are run on the following dataset:

https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2

It follows the schema:

#### 4. Find the number of games where the guessed language and target language is Maltese.

In [ ]:
%%sparksql
SELECT count(*)
FROM dataset
WHERE dataset.guess = dataset.target AND dataset.guess == "Maltese"

#### 5. Return the number of distinct "target" languages.

In [ ]:
%%sparksql
SELECT count(*)
FROM (
    SELECT DISTINCT dataset.target
    FROM dataset
)

#### 6. Return the sample IDs (i.e., the "sample" field) of the first three games where the guessed language is correct (equal to the target one) ordered by date (descending), then by language (ascending), then by country (descending).

In [ ]:
%%sparksql
SELECT sample
FROM dataset
WHERE dataset.guess == dataset.target
ORDER BY dataset.date DESC, dataset.guess ASC, dataset.country DESC
LIMIT 3

#### 7. Aggregate all games by country and "guess" language, counting the number of guesses for each group and return the frequencies of the two most frequent country/language combinations.

In [ ]:
%%sparksql
SELECT dataset.country, dataset.guess, count(*) AS count
FROM dataset
GROUP BY dataset.country, dataset.guess
ORDER BY count DESC
LIMIT 2

#### 8. Sort the languages by decreasing overall percentage of correct guesses and return the first four languages.

In [ ]:
%%sparksql
WITH total_targets AS (
    SELECT dataset.target AS target, count(*) AS total
    FROM dataset
    GROUP BY target
    ),
correct_guesses AS (
    SELECT dataset.guess AS guess, count(*) AS correct
    FROM dataset
    WHERE dataset.guess == dataset.target
    GROUP BY guess
    )
SELECT total_targets.target, correct_guesses.correct / total_targets.total AS percentage
FROM total_targets
JOIN correct_guesses ON correct_guesses.guess == total_targets.target
ORDER BY percentage DESC
LIMIT 3